In [ ]:
#Transformar Biosignals .txt formato a Formato .json para EdgeImpulse:
import json

def parse_opensignals_text_file(filename):
    header_info = {}
    data = []

    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('# {'):
                # Extract device identifier from header line
                start_index = line.find('"') + 1
                end_index = line.find('":', start_index)
                device_id = line[start_index:end_index]
                
                # Parse header info from JSON
                header_info = json.loads(line[2:])
            elif not line.startswith('#'):  # Ignore comment lines
                data.append(list(map(float, line.strip().split())))

    return device_id, header_info, data

def convert_to_desired_json(device_id, header_info, data):
    # Extract relevant information from header_info
    device_name = header_info.get(device_id, {}).get("device name", "")
    device_type = header_info.get(device_id, {}).get("device", "")
    sampling_rate = header_info.get(device_id, {}).get("sampling rate", 1000)
    columns = header_info.get(device_id, {}).get("column", [])
    
    # Calculate interval_ms
    interval_ms = 1000.0 / sampling_rate if sampling_rate != 0 else 0.0
    
    # Prepare sensors list from columns (excluding 'nSeq' column)
    sensors = [{"name": col, "units": "ADC"} for col in columns[5:]]  # Assuming units are "ADC"
    
    # Prepare values list from data
    values = []
    for row in data:
        if len(row) != len(columns):
            raise ValueError("Number of columns in data does not match columns in header_info")
        values.append([float(val) for val in row[5:]])  # Extract values from the 6th column onwards
    
    # Check if number of rows in values matches number of rows in data
    if len(values) != len(data):
        raise ValueError("Number of rows in values should match number of rows in data")
    
    # Construct the desired JSON structure
    json_data = {
        "payload": {
            "device_name": device_name,
            "device_type": device_type,
            "interval_ms": interval_ms,
            "sensors": sensors,
            "values": values
        },
        "protected": {
            "alg": "none",
            "ver": "v1"
        },
        "signature": "00"
    }
    
    return json_data


def write_json_to_file(json_data, output_file):
    with open(output_file, 'w') as file:
        json.dump(json_data, file, indent=4)

if __name__ == "__main__":
    input_filename = "4 - EEG.txt"  # Replace with your actual file path
    output_filename = "Ejercicios_mate.json"  # Replace with desired output file path
    
    # Parse the OpenSignals text file
    device_id, header_info, data = parse_opensignals_text_file(input_filename)
    
    # Convert to the desired JSON format
    json_data = convert_to_desired_json(device_id, header_info, data)
    
    # Write the JSON data to file
    write_json_to_file(json_data, output_filename)
    
    print(f"JSON data has been written to {output_filename}")

#Consideraciones:
#Se consideró únicamente los valores "raw" (ADC) sin pasarlo a mV.
#Se ocnsideró solamente los canales de bitalino que se usaron en la adquisición de datos (A2 para EKG, A1 para EMG y A4 para EEG.
#En el caso del EKG, se acortó la longitud de las 3 señales en un factor de 16 para poder visualizar mejor en el EdgeImpulse.

In [ ]:
#Subida de datos a EdgeImpulse:
#En general, se usó el siguiente código proporcionado por la plataforma Web EdgeImpulse:

# Install requests via: `pip3 install requests`
import requests
import os

api_key = 'ei_2537391015d558ff65133bb634f6537061339d1e9bec9d75549e5263e7226372' #API para comunicarse con el proyecto online. En este caso, esta API es del proyecto EEG.
# Add the files you want to upload to Edge Impulse
files = [
    'Ejercicios_mate.json', #Adaptar para cada señal que se va a subir.
]
# # Replace the label with your own.
label = 'EEG_Mates' #Label para etiquetar las señales.
# Upload the file to Edge Impulse using the API, and print the response.
res = requests.post(url='https://ingestion.edgeimpulse.com/api/training/files',
                    headers={
                        'x-label': label,
                        'x-api-key': api_key,
                    },
                    # Creating the data payload for the request.
                    files=(('data', (os.path.basename(i), open(
                        i, 'rb'), 'image/png')) for i in files)
                    )

if (res.status_code == 200):
    print('Uploaded file(s) to Edge Impulse\n', res.status_code, res.content)
else:
    print('Failed to upload file(s) to Edge Impulse\n',
          res.status_code, res.content)
    